In [ ]:
using LinearAlgebra

Julia has a function `cond` to compute matrix condition numbers. By default, the 2-norm is used. As an example, the family of *Hilbert matrices* is famously badly conditioned. Here is the $7\times 7$  case. 

In [ ]:
A = [ 1/(i+j) for i=1:7, j=1:7 ]

In [ ]:
kappa = cond(A)

Next we engineer a linear system problem to which we know the exact answer.

In [ ]:
x_exact = 1.:7.
b = A*x_exact

Now we perturb the data randomly with a vector of norm $10^{-12}$. 

In [ ]:
dA = randn(size(A));  dA = 1e-12*(dA/norm(dA));
db = randn(size(b));  db = 1e-12*(db/norm(db));

We solve the perturbed problem using built-in pivoted LU and see how the solution was changed.

In [ ]:
x = (A+dA) \ (b+db); 
dx = x - x_exact;

Here is the relative error in the solution.

In [ ]:
rel_error = norm(dx) / norm(x_exact)

And here are upper bounds predicted using the condition number of the original matrix. 

In [ ]:
@show b_bound = kappa * 1e-12/norm(b);
@show A_bound = kappa * 1e-12/norm(A);

Even if we don't make any manual perturbations to the data, machine epsilon does when we solve the linear system numerically.

In [ ]:
x = A\b;
@show rel_error = norm(x - x_exact) / norm(x_exact);
@show rounding_bound = kappa*eps();

Because $\kappa\approx 10^8$, it's possible to lose 8 digits of accuracy in the process of passing from $A$ and $b$ to $x$. That's independent of the algorithm; it's inevitable once the data are expressed in double precision. 

Larger Hilbert matrices are even more poorly conditioned.

In [ ]:
A = [ 1/(i+j) for i=1:14, j=1:14 ];
kappa = cond(A)

Before we compute the solution, note that $\kappa$ exceeds `1/eps()`. In principle we therefore might end up with an answer that is completely wrong (i.e., a relative error greater than 100%).

In [ ]:
rounding_bound = kappa*eps()

In [ ]:
x_exact = 1.:14.;
b = A*x_exact;  
x = A\b;

We got an answer. But in fact the error does exceed 100%.

In [ ]:
relative_error = norm(x_exact - x) / norm(x_exact)